# Concatenation and Pivoting of New Parts Usage

In [1]:
# Import needed Libraries.
from datetime import datetime as dt
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

In [19]:
# Import the raw data from excel files.  This is going to take a minute or five.
report1 = pd.read_excel('report1.xlsx') # 2015 usage part 1
report2 = pd.read_excel('report2.xlsx') # 2015 usage part 2
report3 = pd.read_excel('report3.xlsx') # 2016 usage part 1

In [20]:
# Now let's combine all these into one table.
new_usage = pd.concat([report1,report2,report3], ignore_index=True)
new_usage.shape

(504834, 4)

In [21]:
# Let's look at the top of this frame.
new_usage.head()

,Trans,Item_Number,Qty_Change,Date
0,10000000.0,P1095,4.0,2015-01-04
1,10000001.0,P1063,48.0,2015-01-04
2,10000002.0,P1066,150.0,2015-01-04
3,10000003.0,P1107,100.0,2015-01-04
4,10000004.0,P1083,126.0,2015-01-04


In [22]:
# Now the bottom.
new_usage.tail()

,Trans,Item_Number,Qty_Change,Date
504829,25521147.0,P1330,60.0,2016-06-30
504830,25521148.0,P1081,200.0,2016-06-30
504831,25521149.0,P1914,32.0,2016-06-30
504832,NaN,NaN,NaN,NaT
504833,NaN,REPORT TOTAL,8279154.0,NaT


In [23]:
# Let's get rid of any row that has nan(blank) as the transactions id. 
new_usage = new_usage.dropna(subset=['Trans'])

In [ ]:
# We need to make Trans a string.
new_usage["Trans"] = new_usage["Trans"].astype(int).astype('str')
# new_usage["Line"] = new_usage["Line"].astype(int).astype('str')

In [ ]:
# We want to pivot this by week so we need to add a column for week.
# Since we need this ordered by year also we will add a column for year.
# We will throw in month to make me feel better.

new_usage["Week"] = new_usage["Date"].dt.week
new_usage["Month"] = new_usage["Date"].dt.month
new_usage["Year"] = new_usage["Date"].dt.year

In [ ]:
new_usage.tail()

In [ ]:
tmp_df = new_usage

In [ ]:
# Split the data set up so that we can modify the week number.
week_53 = tmp_df[:][(tmp_df["Year"] == 2016) & (tmp_df["Week"] == 53)]
all_other_2015 = tmp_df[:][(tmp_df["Year"] == 2015)]
all_other_2016 = tmp_df[:][(tmp_df["Year"] == 2016) & (tmp_df["Week"] != 53)]
all_other_2016.head()

In [ ]:
# Get rid of all instances of week 53 from 2016.
week_53["Week"] = 1
# Add a week to the rest of the weeks of 2016.
all_other_2016["Week"] = all_other_2016["Week"] + 1
all_other_2016.head()

In [ ]:
# Put the data sets back together.
tmp_df = pd.concat([week_53, all_other_2015, all_other_2016])
# Check that we still have the same amount of data as our original data set.
if tmp_df.shape == new_usage.shape:
    new_usage = tmp_df
    print('Success!')
else:
    print('new: {}, original: {}').format(tmp_df.shape, new_usage.shape)

In [ ]:
# Pivot the data grouping the items and displaying the usage by week.
pivoted = pd.pivot_table(tmp_df, values="Qty Change", index="Item Number", columns=["Year", "Week"], aggfunc=np.sum)
pivoted.head()

In [ ]:
# Save the pivoted data back to an Excel file.
pivoted.to_excel('pivoted_new.xlsx', sheet_name="pivot", na_rep=0)

In [ ]:
# Save the core usage to Excel. This has the week 53 in Jan, 2016.
new_usage.to_excel('new_usage.xlsx', na_rep=0, index=False)
# Save the corrected data back to Excel. This does not have week 53 in Jan, 2016.
# tmp_df.to_excel('tmp_df.xlsx', na_rep=0)